In [ ]:
import numpy as np
import pandas as pd

**Data extraction with Kaggle API**

In [ ]:
#dit kan wel weg denk ik
!pip install kaggle

In [22]:
#every other person except me should also put their kaggle file in a hidden .kaggle dir
#i think this is still okay even if i use in vsc
#put the kaggle file in gitignore

import os
import shutil

#source = "/Users/lodedockx/Documents/Kaggle/kaggle.json"
#destination = os.path.expanduser("~/.kaggle/kaggle.json")

#create .kaggle folder if not exists
#os.makedirs(os.path.dirname(destination),exist_ok=True)

#move the file
#shutil.move(source,destination)

#print("kaggle.json moved successfully")


In [17]:
!kaggle datasets download -d girardi69/marathon-time-predictions

Dataset URL: https://www.kaggle.com/datasets/girardi69/marathon-time-predictions
License(s): CC0-1.0
  0% 0.00/2.63k [00:00<?, ?B/s]
100% 2.63k/2.63k [00:00<00:00, 10.4MB/s]


In [18]:
import zipfile

with zipfile.ZipFile("marathon-time-predictions.zip","r") as zip_ref:
    zip_ref.extractall("marathon-time-predictions")

In [19]:
print(os.listdir("marathon-time-predictions"))

['MarathonData.csv']


In [23]:
df = pd.read_csv("marathon-time-predictions/MarathonData.csv")

In [24]:
df

,id,Marathon,Name,Category,km4week,sp4week,CrossTraining,Wall21,MarathonTime,CATEGORY
0,1,Prague17,Blair MORGAN,MAM,132.8,14.434783,NaN,1.16,2.37,A
1,2,Prague17,Robert Heczko,MAM,68.6,13.674419,NaN,1.23,2.59,A
2,3,Prague17,Michon Jerome,MAM,82.7,13.520436,NaN,1.30,2.66,A
3,4,Prague17,Daniel Or lek,M45,137.5,12.258544,NaN,1.32,2.68,A
4,5,Prague17,Luk ? Mr zek,MAM,84.6,13.945055,NaN,1.36,2.74,A
...,...,...,...,...,...,...,...,...,...,...
82,83,Prague17,Stefano Vegliani,M55,50.0,10.830325,NaN,2.02,3.93,D
83,84,Prague17,Andrej Madliak,M40,33.6,10.130653,ciclista 3h,1.94,3.93,D
84,85,Prague17,Yoi Ohsako,M40,55.4,11.043189,NaN,1.94,3.94,D
85,86,Prague17,Simon Dunn,M45,33.2,11.066667,NaN,2.05,3.95,D


# Data preprocessing

To Do:
1. exclude: id, Marathon, Name, and CATEGORY (last column) from the dataset. No relevance and last column gives more info about the performance so should not be in the model

2. one hot encoding: Category, CrossTraining

1. Exclude columns

In [ ]:
#1. Exclude columns
columns_to_drop = ['id', 'Marathon', 'Name', 'CATEGORY']

for col in columns_to_drop:
    df = df.drop(col, axis=1)

In [ ]:
df.head(87)

,Category,km4week,sp4week,CrossTraining,Wall21,MarathonTime
0,MAM,132.8,14.434783,NaN,1.16,2.37
1,MAM,68.6,13.674419,NaN,1.23,2.59
2,MAM,82.7,13.520436,NaN,1.30,2.66
3,M45,137.5,12.258544,NaN,1.32,2.68
4,MAM,84.6,13.945055,NaN,1.36,2.74
...,...,...,...,...,...,...
82,M55,50.0,10.830325,NaN,2.02,3.93
83,M40,33.6,10.130653,ciclista 3h,1.94,3.93
84,M40,55.4,11.043189,NaN,1.94,3.94
85,M45,33.2,11.066667,NaN,2.05,3.95


In [ ]:
#2. one hot encoding

#only select the columns that need to be encoded
to_be_encoded_df = df['Category']
#rest df
df = df.drop('Category', axis=1)

#encode
df_encoded = pd.get_dummies(to_be_encoded_df, dtype=int)

In [ ]:
df_merged = pd.concat([df, df_encoded], axis=1)

In [ ]:
df_merged

,km4week,sp4week,CrossTraining,Wall21,MarathonTime,M40,M45,M50,M55,MAM,WAM
0,132.8,14.434783,NaN,1.16,2.37,0,0,0,0,1,0
1,68.6,13.674419,NaN,1.23,2.59,0,0,0,0,1,0
2,82.7,13.520436,NaN,1.30,2.66,0,0,0,0,1,0
3,137.5,12.258544,NaN,1.32,2.68,0,1,0,0,0,0
4,84.6,13.945055,NaN,1.36,2.74,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
82,50.0,10.830325,NaN,2.02,3.93,0,0,0,1,0,0
83,33.6,10.130653,ciclista 3h,1.94,3.93,1,0,0,0,0,0
84,55.4,11.043189,NaN,1.94,3.94,1,0,0,0,0,0
85,33.2,11.066667,NaN,2.05,3.95,0,1,0,0,0,0


In [ ]:
#at the moment, this excludes the type of crosstraining
df_merged["CrossTraining"] = (
    df_merged["CrossTraining"]
    .str.extract(r"(\d+)")
    .astype(float)
    .fillna(0)
)

In [ ]:
df_merged

,km4week,sp4week,CrossTraining,Wall21,MarathonTime,M40,M45,M50,M55,MAM,WAM
0,132.8,14.434783,0.0,1.16,2.37,0,0,0,0,1,0
1,68.6,13.674419,0.0,1.23,2.59,0,0,0,0,1,0
2,82.7,13.520436,0.0,1.30,2.66,0,0,0,0,1,0
3,137.5,12.258544,0.0,1.32,2.68,0,1,0,0,0,0
4,84.6,13.945055,0.0,1.36,2.74,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
82,50.0,10.830325,0.0,2.02,3.93,0,0,0,1,0,0
83,33.6,10.130653,3.0,1.94,3.93,1,0,0,0,0,0
84,55.4,11.043189,0.0,1.94,3.94,1,0,0,0,0,0
85,33.2,11.066667,0.0,2.05,3.95,0,1,0,0,0,0


In [ ]:
#ensure numeric fields are numeric
for col in ["km4week", "sp4week", "CrossTraining", "Wall21"]:
    df_merged[col] = pd.to_numeric(df_merged[col], errors="coerce")

In [ ]:
#fill all the missing values with medians
df_merged = df_merged.fillna(df_merged.median(numeric_only=True))

In [ ]:
df_merged

,km4week,sp4week,CrossTraining,Wall21,MarathonTime,M40,M45,M50,M55,MAM,WAM
0,132.8,14.434783,0.0,1.16,2.37,0,0,0,0,1,0
1,68.6,13.674419,0.0,1.23,2.59,0,0,0,0,1,0
2,82.7,13.520436,0.0,1.30,2.66,0,0,0,0,1,0
3,137.5,12.258544,0.0,1.32,2.68,0,1,0,0,0,0
4,84.6,13.945055,0.0,1.36,2.74,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
82,50.0,10.830325,0.0,2.02,3.93,0,0,0,1,0,0
83,33.6,10.130653,3.0,1.94,3.93,1,0,0,0,0,0
84,55.4,11.043189,0.0,1.94,3.94,1,0,0,0,0,0
85,33.2,11.066667,0.0,2.05,3.95,0,1,0,0,0,0


In [ ]:
#feature scaling
from sklearn.preprocessing import StandardScaler

num_cols = ["km4week", "sp4week", "CrossTraining", "Wall21"]
df_merged[num_cols] = StandardScaler().fit_transform(df_merged[num_cols])

In [ ]:
df_merged

,km4week,sp4week,CrossTraining,Wall21,MarathonTime,M40,M45,M50,M55,MAM,WAM
0,2.628775,-0.105867,-0.347277,-2.327754,2.37,0,0,0,0,1,0
1,0.233311,-0.106509,-0.347277,-1.980982,2.59,0,0,0,0,1,0
2,0.759417,-0.106639,-0.347277,-1.634211,2.66,0,0,0,0,1,0
3,2.804144,-0.107704,-0.347277,-1.535133,2.68,0,1,0,0,0,0
4,0.830311,-0.106280,-0.347277,-1.336978,2.74,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
82,-0.460702,-0.108910,-0.347277,1.932582,3.93,0,0,0,1,0,0
83,-1.072628,-0.109501,1.242886,1.536272,3.93,1,0,0,0,0,0
84,-0.259215,-0.108730,-0.347277,1.536272,3.94,1,0,0,0,0,0
85,-1.087553,-0.108710,-0.347277,2.081199,3.95,0,1,0,0,0,0


# Splitting Data

In [ ]:
#splitting the data
from sklearn.model_selection import train_test_split

X = df_merged.drop('MarathonTime',axis=1)
y = df_merged['MarathonTime']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=8)

# Training Models

RandomForest,
XGBoost,
Linear Regression

In [ ]:
from sklearn.model_selection import cross_val_score

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#initialize the model
lr_model = LinearRegression()

#fit the model
lr_model.fit(X_train,y_train)

#predict y
lr_y_predictions = lr_model.predict(X_test)

#show predictions
lr_y_predictions

#very naive without CV, still need to implement

array([3.79368065, 3.44941357, 3.32400403, 2.80530981, 3.14321323,
       2.6167968 , 3.09035185, 3.36611207, 3.75121605, 2.39982413,
       3.46406153, 3.23351927, 2.69371743, 3.28911102, 3.65032466,
       3.88431473, 2.91550381, 3.12170214, 3.20584014, 3.45677817,
       3.1962406 , 3.21065387])

In [ ]:
#MODEL PERFORMANCE
#mean squared error
mean_squared_error(y_test,lr_y_predictions)

#cv score
#print(cross_val_score(lr_model,X_train, y_train, cv=2)) # --> this is saying nothing to me

0.018129993794126772

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#init the model
rf_model = RandomForestRegressor()

#fit the model
rf_model.fit(X_train, y_train)

#predict y
rf_y_predictions = rf_model.predict(X_test)

#show predictions
rf_y_predictions

array([3.7072, 3.3194, 3.4306, 2.7922, 3.188 , 2.615 , 3.1476, 3.552 ,
       3.8487, 2.9186, 3.4721, 3.457 , 2.7312, 3.3008, 3.6347, 3.8837,
       2.9073, 3.1532, 3.2803, 3.5148, 3.3424, 3.2657])

In [ ]:
#model performance
mean_squared_error(y_test,rf_y_predictions)

0.016478999545454526

## XG Boost

In [ ]:
from xgboost import XGBRegressor

#init the model
xgb_model = XGBRegressor()

#fit the model
xgb_model.fit(X_train, y_train)

#predict y
xgb_y_predictions = xgb_model.predict(X_test)

#show predictions
xgb_y_predictions

array([3.7351336, 3.2777138, 3.4090047, 2.8633842, 3.1760626, 2.6449385,
       2.924158 , 3.5619931, 3.8723373, 2.893333 , 3.5000482, 3.4345512,
       2.5577207, 3.2823505, 3.542551 , 3.8829412, 2.9562273, 3.098702 ,
       3.213737 , 3.4505072, 3.2112575, 3.204875 ], dtype=float32)

In [ ]:
xgb_model = XGBRegressor()

In [ ]:
X_test